# Fire Distance Testing on Lyons

In [1]:
import evacnx as ex
import osmnx as ox
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
import geopandas as gpd
import contextily as cx
import time

import numpy as np
import networkx as nx
import math
import time
from scipy.interpolate import UnivariateSpline
import osmnx as ox
import matplotlib.pyplot as plt
from pyproj import CRS, Transformer
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.geometry import LineString
from shapely.geometry import MultiLineString
from shapely.ops import transform
from shapely.ops import unary_union
from networkx.algorithms.flow import shortest_augmenting_path
import geopandas as gpd
import contextily as cx

In [2]:
location = 'Lyons Colorado'
sup_dem_nodes = [(1,-4000),(30,4000),(4,-4000),(40,4000)]
G = ex.construct_orig_graph(location,sup_dem_nodes)
fire_origin_radii = (40.227381,-105.261359,0.05)

In [3]:
def geodesic_point_buffer(graph, lat, lon, radius):
    """
    Creates circles from given latitude, longitude, and radius
    :param lat: latitude from original data
    :param lon: longitude from original data
    :param radius: radius from original data
    """
    aeqd_proj = CRS.from_proj4(f"+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0")
    tfmr = Transformer.from_proj(aeqd_proj, aeqd_proj.geodetic_crs)
    buf = Point(0, 0).buffer(radius * 1609.34)  # distance in miles (converts from meters to miles)
    circle = Polygon(transform(tfmr.transform, buf).exterior.coords[:])
    net_trans = Transformer.from_crs(aeqd_proj.geodetic_crs, graph.graph['crs'])
    proj_circ = Polygon(transform(net_trans.transform,circle).exterior.coords[:])
    
    return proj_circ

In [4]:
def dist_to_fire(G, fire_polygon, **kwargs):
    smoke_polygon = kwargs.get('smoke_polygon',fire_polygon)
    ###find nodes inside poly to remove
    start_gs = time.process_time()
    gs_nodes = ox.utils_graph.graph_to_gdfs(G,nodes = True,edges = False, node_geometry = True)[['geometry']]
    gs_edges = ox.utils_graph.graph_to_gdfs(G,nodes = False, edges = True)[['geometry']]
    end_gs = time.process_time()

    nodes_removed = []
    edge_dist = {}
    start_to_remove = time.process_time()
    for point in gs_nodes['geometry']:
        if point.intersects(smoke_polygon):
            node = ox.distance.nearest_nodes(G,point.x,point.y,return_dist=False)
            nodes_removed.append(node)
    for i in range(len(gs_edges)):
        entry =  gs_edges.iloc[i]
        dist = entry['geometry'].distance(fire_polygon)
        entry = gs_edges.iloc[[i]]
        edge = entry['geometry'].index[0]
        if G.is_multigraph():
            # G[edge[0]][edge[1]][edge[2]]['fire_dist'] = dist
            edge_dist[(edge[0],edge[1],edge[2])] = dist
        elif G.is_directed():
            # G[edge[0]][edge[1]]['fire_dist'] = dist
            edge_dist[(edge[0],edge[1])] = dist
        else:
            raise Exception("Must Input a DiGraph or MultiDiGraph")
    end_to_remove = time.process_time()

    return (nodes_removed, edge_dist)

In [5]:
def create_fire_mat(G, fire_origins_radii, time_horizon, removed_node_mat, edge_dist_mat, fire_polygon_mat,**kwargs):
    removed_node_mat_copy = removed_node_mat.copy()
    edge_dist_mat_copy = edge_dist_mat.copy()
    fire_polygon_mat_copy = fire_polygon_mat.copy()
    
    start = time.time()
    time_ints_done = kwargs.get('start_time_int',len(removed_node_mat))
    shape = kwargs.get('shape', 'square')
    

    if shape == 'circle':
        fire_lat = fire_origins_radii[0]
        fire_long = fire_origins_radii[1]
        init_radius = fire_origins_radii[2]
        fire_polygon = geodesic_point_buffer(G,fire_lat,fire_long,init_radius)
            
    else:
        # (,0.09),(,0.09),(,0.07)
        coords = ((40.231862,-105.266485), (40.228941,-105.264405), (40.233535,-105.262882), (40.227381,-105.261359))
        fire_polygon = Polygon(coords)
        
    for i in range(time_horizon):
        nodes_removed, edge_distances = dist_to_fire(G, fire_polygon)

        # fire_polygon_mat_copy.append(fire_polygon)
        removed_node_mat_copy.append(nodes_removed)
        edge_dist_mat_copy.append(edge_distances)
        
    end = time.time()
    print(f"Time to do Fire Poly for Time Horizon {time_horizon}: {end - start}")
    
    return end - start

In [7]:
times = []
for i in range(0,20):
    num_time_ints = 100
    rmvd_nodes_mat = []
    edge_dist_mat = []
    fire_polygon_mat = []
    comp_time = create_fire_mat(G, fire_origin_radii, num_time_ints, rmvd_nodes_mat, edge_dist_mat, fire_polygon_mat,shape = 'circle')
    times.append(comp_time)
times_np = np.array(times)
average = np.mean(times_np)
print(average)

Time to do Fire Poly for Time Horizon 100: 18.932679653167725
Time to do Fire Poly for Time Horizon 100: 18.9295814037323
Time to do Fire Poly for Time Horizon 100: 18.997164726257324
Time to do Fire Poly for Time Horizon 100: 19.08865976333618
Time to do Fire Poly for Time Horizon 100: 19.093363285064697
Time to do Fire Poly for Time Horizon 100: 18.987788915634155
Time to do Fire Poly for Time Horizon 100: 18.929175853729248
Time to do Fire Poly for Time Horizon 100: 19.180400609970093
Time to do Fire Poly for Time Horizon 100: 19.10226273536682
Time to do Fire Poly for Time Horizon 100: 18.97879385948181
Time to do Fire Poly for Time Horizon 100: 19.005411863327026
Time to do Fire Poly for Time Horizon 100: 18.915698528289795
Time to do Fire Poly for Time Horizon 100: 19.020750045776367
Time to do Fire Poly for Time Horizon 100: 18.921380281448364
Time to do Fire Poly for Time Horizon 100: 18.873543739318848
Time to do Fire Poly for Time Horizon 100: 18.90126085281372
Time to do Fir

In [8]:
times = []
for i in range(0,20):
    num_time_ints = 100
    rmvd_nodes_mat = []
    edge_dist_mat = []
    fire_polygon_mat = []
    comp_time = create_fire_mat(G, fire_origin_radii, num_time_ints, rmvd_nodes_mat, edge_dist_mat, fire_polygon_mat)
    times.append(comp_time)
times_np = np.array(times)
average = np.mean(times_np)
print(average)

Time to do Fire Poly for Time Horizon 100: 19.667536735534668
Time to do Fire Poly for Time Horizon 100: 19.90530300140381
Time to do Fire Poly for Time Horizon 100: 19.128070831298828
Time to do Fire Poly for Time Horizon 100: 23.25837755203247
Time to do Fire Poly for Time Horizon 100: 21.217743635177612
Time to do Fire Poly for Time Horizon 100: 19.15635585784912
Time to do Fire Poly for Time Horizon 100: 18.758914709091187
Time to do Fire Poly for Time Horizon 100: 18.60014772415161
Time to do Fire Poly for Time Horizon 100: 18.611605644226074
Time to do Fire Poly for Time Horizon 100: 18.68834090232849
Time to do Fire Poly for Time Horizon 100: 18.722384691238403
Time to do Fire Poly for Time Horizon 100: 18.72345495223999
Time to do Fire Poly for Time Horizon 100: 18.70154881477356
Time to do Fire Poly for Time Horizon 100: 18.69614839553833
Time to do Fire Poly for Time Horizon 100: 18.807191848754883
Time to do Fire Poly for Time Horizon 100: 18.685792684555054
Time to do Fire 